# Building a chat bot without 3rd party libraries

### Task

To build a narrow domain chatbot without using any 3rd party APIs which functions as a coffee sales executive at starbucks.

### About the demo

This demo uses the [rasa_nlu](https://github.com/RasaHQ/rasa_nlu) library (with spacy as backend) for the bot. I did try a few other ways to implement a chatbot, but nothing worked as well as this did.

### Rasa NLU

Rasa NLU (Natural Language Understanding) is a tool for intent classification and entity extraction. You can think of Rasa NLU as a set of high level APIs for building your own language parser using existing NLP and ML libraries.

### Training data

The training data is stored in "coffee-training-data.json" by myself. This have very few queries and needs to be much more diverse range of human queries. 

### Status of project

This is still a basic implementation with every little training data. This bot can be improved by: (i) giving more training data, (ii) writing advanced functions to handle the results from rasa_nlu.

## Importing required functions from Rasa NLU

In [1]:
from rasa_nlu.converters import load_data
from rasa_nlu.config import RasaNLUConfig
from rasa_nlu.model import Trainer

## Training using the data from "coffee-training-data.json"

In [2]:
training_data = load_data('rasa_nlu/coffee-training-data.json')
trainer = Trainer(RasaNLUConfig("rasa_nlu/config_spacy.json"))
trainer.train(training_data)
model_directory = trainer.persist('./models/')  # Returns the directory the model is stored in

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.1s finished


## Loading the model so that it can be used

In [3]:
from rasa_nlu.model import Metadata, Interpreter

metadata = Metadata.load(model_directory)   # where model_directory points to the folder the model is persisted in
interpreter = Interpreter.load(metadata, RasaNLUConfig("rasa_nlu/config_spacy.json"))

## Seeing sample response of model for given input

In [4]:
k = interpreter.parse(u"Can I have a latte?")
print(k)

{u'entities': [{u'start': 13, u'extractor': u'ner_crf', u'end': 18, u'value': u'latte', u'entity': 'item'}], u'intent': {u'confidence': 0.53606092844861997, u'name': u'order_details'}, u'text': u'Can I have a latte?', u'intent_ranking': [{u'confidence': 0.53606092844861997, u'name': u'order_details'}, {u'confidence': 0.27618444869366371, u'name': u'goodbye'}, {u'confidence': 0.1321524262437597, u'name': u'greet'}, {u'confidence': 0.055602196613956677, u'name': u'affirm'}]}


## Understanding what the user is saying

In [5]:
print("*User says:* " + k['text'])
print("When some says the above, here's the understanding of the bot.")
print("  " + "The user's intent is: " + k['intent']['name'] + " with a confidence of " + str(k['intent']['confidence']) \
      + ", and he's trying to order a " + k['entities'][0]['value'])

*User says:* Can I have a latte?
When some says the above, here's the understanding of the bot.
  The user's intent is: order_details with a confidence of 0.536060928449, and he's trying to order a latte


## Building the function for responding to a query

In [6]:
def coffee_bot(string):
    
    # Printing user's query
    print("Me: " + string)
    
    # Storing the dict response form the bot
    bot_response = interpreter.parse(string.decode('utf-8') )
    
    # Finding user's intent
    user_intent = bot_response['intent']['name']
    
    # Response of bot based on intent of user
    if(user_intent == u'greet'):
        print("Alex: Hi! I hope you're having a good day, what do you to have today?")
    elif(user_intent == u'order_details'):
        order_item = k['entities'][0]['value']
        print("Alex: Thank you for ordering a " + order_item + ". It will be delivered in a few minutes.")
    elif(user_intent == u'goodbye'):
        print("Alex: It was nice talking to you, have a nice day! Goodbye.")
    
    return 

## DEMO: Talking to the bot

In [7]:
coffee_bot("Hello!")
coffee_bot("Can I have a latte?")
coffee_bot("Bye.")

Me: Hello!
Alex: Hi! I hope you're having a good day, what do you to have today?
Me: Can I have a latte?
Alex: Thank you for ordering a latte. It will be delivered in a few minutes.
Me: Bye
Alex: It was nice talking to you, have a nice day! Goodbye.


In [10]:
coffee_bot("Yo!")
coffee_bot("I need an latte.")
coffee_bot("Bye.")

Me: Yo!
Alex: Hi! I hope you're having a good day, what do you to have today?
Me: I need an latte.
Alex: Thank you for ordering a latte. It will be delivered in a few minutes.
Me: Bye.
Alex: Hi! I hope you're having a good day, what do you to have today?
